In [1]:
import pandas as pd
import numpy as np
import cmath
import math
from scipy.optimize import fmin, minimize
from astropy import units as u
from scipy.interpolate import RegularGridInterpolator
import matplotlib.pyplot as plt
import copy
%matplotlib inline

DATA STUFF

In [2]:
"""
Data class
"""
class data:
    u: float
    v: float
    phase: float
    amp: float
    sigma: float
    vis_data: complex
    def __init__(self, u, v, phase, amp, sigma):
        self.u = u
        self.v = v
        self.phase = phase
        self.amp = amp
        self.sigma = sigma
        self.vis_data = amp * np.exp(1j * math.radians(phase))

    def __repr__(self):
        return f"[u: {self.u}, v: {self.v}]"

    def __str__(self):
        return f"[u: {self.u}, v: {self.v}]"

In [ ]:
def process_data(data_df):
    coords = []
    data_list = []
    for i in range(len(data_df)):
        data_list.append(data(data_df.loc[i, 'U(lambda)'], data_df.loc[i, 'V(lambda)'], data_df.loc[i, 'Iphase(d)'], data_df.loc[i, 'Iamp(Jy)'], data_df.loc[i, 'Isigma(Jy)']))
        coords.append([data_df.loc[i, 'U(lambda)'], data_df.loc[i, 'V(lambda)']])
    coords = np.array(coords)
    return coords, data_list

In [ ]:
def read_data(filename: str()):
    df = pd.read_csv(filename)
    return df

In [ ]:
df = read_data("./data/SR1_M87_2017_095_hi_hops_netcal_StokesI.csv")
coords, data_list = process_data(df)
len(data_list)

INTERPOLATION AND REGULARIZATION

In [31]:
# Assumption image is 80x80 pixels
# Pass in array of u,v coords then return array of interpolated values
def interpolate(image, coords, FOV):
    ft_image = np.fft.fftshift(np.fft.fft2(image)) # Investigate to make sure what this fft does

    #kx_1 = 2*math.pi/FOV
    kx_1 = 1/FOV
    # kx_1 is 1/coeff

    kx = np.linspace(ft_image.shape[0]//-2, (ft_image.shape[0])//2 - 1, ft_image.shape[0]) * kx_1
    ky = np.linspace(ft_image.shape[0]//-2, (ft_image.shape[1])//2 - 1, ft_image.shape[1]) * kx_1
    
    interp_real = RegularGridInterpolator((kx, ky), ft_image.real, bounds_error=False, method="linear")
    interp_imag = RegularGridInterpolator((kx, ky), ft_image.imag, bounds_error=False, method="linear")

    real = interp_real(coords)
    imag = interp_imag(coords)
    #print(imag)
    return real + imag * 1j

In [9]:
# p is the exponent of the regularizing terms
# The smaller p the more sensitive it is to noise
# TSV = Total Squared Variation
def calc_regularizer(image: np.array, tsv=False, p=None):
    if tsv and p == None:
        raise Exception("p value not set")
    reg = 0
    if tsv:
        image_lshift = np.copy(image, subok=True)
        image_lshift = np.roll(image_lshift, -1,axis=1)
        image_lshift[:,-1] = 0
        image_upshift = np.copy(image, subok=True)
        image_upshift = np.roll(image_upshift, -1, axis=0)
        image_upshift[-1] = 0

        term_1 = np.power(np.absolute(np.subtract(image_lshift, image)),p)
        term_2 = np.power(np.absolute(np.subtract(image_upshift, image)),p)
        reg = np.sum(np.add(term_1,term_2))
    return reg

In [10]:
def gradient_regularizer(image: np.array):
    image_lshift = np.copy(image, subok=True)
    image_lshift = np.roll(image_lshift, -1,axis=1)
    image_lshift[:,-1] = 0
    image_upshift = np.copy(image, subok=True)
    image_upshift = np.roll(image_upshift, -1, axis=0)
    image_upshift[-1] = 0
    image_rshift = np.copy(image, subok=True)
    image_rshift = np.roll(image_rshift, 1,axis=1)
    image_rshift[:,0] = 0
    image_dshift = np.copy(image, subok=True)
    image_dshift = np.roll(image_dshift, 1, axis=0)
    image_dshift[0] = 0
    g_reg = 4 * image - image_lshift - image_upshift - image_rshift - image_dshift
    return g_reg

LOSS FUNCTION

In [11]:
# Assumption image is 80px by 80px => 6400 variables
def loss(image, data_list: list[data], coords, p = 2, reg_weight = 1, FOV = 100*u.uas.to(u.rad)): #Ask about u.rad
    error_sum = 0
    vis_images = interpolate(image, coords, FOV)

    for i in range(len(data_list)):
        vis_data = data_list[i].amp * np.exp(1j * math.radians(data_list[i].phase))
        vis_image = vis_images[i]
        error = (abs(vis_image-vis_data) / data_list[i].sigma) ** 2
        error_sum += error
    
    return error_sum + reg_weight * calc_regularizer(image=image, tsv=True, p=2)

GRADIENTS

In [13]:
def gradient(data_list: list[data], coords, image, mode = 1): # 0 For central, -1 for backward, 1 for forward
    image_copy = np.copy(image, subok=True)
    upper_diff: float
    lower_diff: float
    h: float
    gradient_arr = np.empty(np.shape(image),dtype=np.complex_)
    if (mode == 0): # Central difference
        for row in range(len(image)):
            for col in range(len(image[row])):
                image_copy[row,col] += 1e-6 / 2
                upper_diff = loss(image_copy, data_list, coords)
                image_copy[row,col] -= 1e-6
                lower_diff = loss(image_copy, data_list, coords)
                image_copy[row,col] = image[row,col] # Reset that pixel to original value
                gradient_arr[row,col] = (upper_diff - lower_diff) / 1e-6
    elif (mode == -1): # Backward difference
        upper_diff = loss(image, data_list, coords)
        for row in range(len(image)):
            for col in range(len(image[row])):
                image_copy[row,col] -= 1e-8 # Check with Misha 
                lower_diff = loss(image_copy, data_list, coords)
                gradient_arr[row,col] = (upper_diff - lower_diff) / 1e-8
                image_copy[row,col] = image[row,col]
    elif (mode == 1) : # Forward difference is default
        lower_diff = loss(image, data_list, coords)
        for row in range(len(image)):
            for col in range(len(image[row])):
                image_copy[row,col] += 1e-8 # Check with Misha 
                upper_diff = loss(image_copy, data_list, coords)
                gradient_arr[row,col] = (upper_diff - lower_diff) / 1e-8
                image_copy[row,col] = image[row,col]
    else:
        raise ValueError('Incorrect mode for finite differences')
    return gradient_arr
    #Return a Gradient 2d array?

In [ ]:
emp = np.zeros((80,80))
x = gradient(data_list, coords, emp)
np.savetxt("gradient_forward.txt", np.real(x), delimiter=",")

In [18]:
def gradient_2(data_list: list[data], coords, coeffs, image, FOV = 100*u.uas.to(u.rad)):
    gradient_arr = np.empty(np.shape(image)) # Because we are in real space
    vis_images = interpolate(image, coords, FOV)
    for row in range(len(image)):
        for col in range(len(image[row])):
            gradient_sum = 0
            for i in range(len(data_list)):
                vis_data = data_list[i].vis_data
                vis_image = vis_images[i]
                term_1 = coeffs[row,col,i,0] * (np.conj(vis_image) - np.conj(vis_data))
                term_2 = coeffs[row,col,i,1] * (vis_image - vis_data)
                gradient_sum += (term_1 + term_2)/(data_list[i].sigma ** 2)
            gradient_arr[row,col] = gradient_sum
    return gradient_arr

In [19]:
def preprocess_gradient(data_list: list[data], coords, image, FOV = 100*u.uas.to(u.rad)): # Add FOV here
    r, c = np.shape(image)
    preprocessed = np.empty([r,c,len(data_list),2], dtype=np.complex_)
    for row in range(len(image)):
        for col in range(len(image[row])):
            for datum in range(len(data_list)):
                term = ((2*np.pi*1j)/image.size)*(row*(data_list[datum].u) + col*(data_list[datum].v)) #.size for numpy array returns # of rows * # of cols
                term_1 = np.exp(term)
                term_2 = np.exp(-1*term)
                preprocessed[row,col,datum,0] = term_1
                preprocessed[row,col,datum,1] = term_2
    return preprocessed

In [44]:
emp = np.zeros((80,80))
coeffs = preprocess_gradient(data_list, coords, emp)

In [26]:
coeffs

array([[[[ 1.        +0.00000000e+00j,  1.        +0.00000000e+00j],
         [ 1.        +0.00000000e+00j,  1.        +0.00000000e+00j],
         [ 1.        +0.00000000e+00j,  1.        +0.00000000e+00j],
         ...,
         [ 1.        +0.00000000e+00j,  1.        +0.00000000e+00j],
         [ 1.        +0.00000000e+00j,  1.        +0.00000000e+00j],
         [ 1.        +0.00000000e+00j,  1.        +0.00000000e+00j]],

        [[ 0.72896863-6.84547106e-01j,  0.72896863+6.84547106e-01j],
         [-0.42577929-9.04827052e-01j, -0.42577929+9.04827052e-01j],
         [ 0.79075635-6.12131026e-01j,  0.79075635+6.12131026e-01j],
         ...,
         [ 0.46619364+8.84682704e-01j,  0.46619364-8.84682704e-01j],
         [-0.68724289+7.26427708e-01j, -0.68724289-7.26427708e-01j],
         [ 0.32205916+9.46719545e-01j,  0.32205916-9.46719545e-01j]],

        [[ 0.06279052-9.98026728e-01j,  0.06279052+9.98026728e-01j],
         [-0.63742399+7.70513243e-01j, -0.63742399-7.70513243e-01j],
  

In [45]:
x = gradient_2(data_list, coords, coeffs, emp)
np.savetxt("gradient_2.txt", np.real(x), delimiter=",")

[0. 0. 0. ... 0. 0. 0.]


/var/folders/_b/trlmhkgj5xq968yccj4vtg1c0000gn/T/ipykernel_4165/2184829813.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  gradient_arr[row,col] = gradient_sum


In [ ]:
GRADIENT DESCENT CODE

In [20]:
def gradient_descent(image, data_list, coords, coeffs, subset_percent = 10):
    image_copy = np.copy(image, subok=True)
    # coeffs = preprocess_gradient(data_list, coords, image_copy)
    # loss needs same subset as gradient_2
    i = 0
    grad = None
    t = 10000000
    while grad is None or np.any(np.abs(grad) > 0.0000001): # Didn't finish before so never negative?
        plt.figure()
        plt.imshow(image_copy, vmin=0, vmax=0.5)
        plt.axis('off')
        plt.savefig('images/'+str(i)+'.png')

        #selection = np.random.choice(np.arange(len(data_list)), size=len(coords)*subset_percent//100, replace=False) # selection is full of indicies
        #subset = [data_list[index] for index in selection]
        prev_loss = loss(image_copy, data_list, coords)

        grad = gradient_2(data_list, coords, coeffs, image_copy) + gradient_regularizer(image_copy)
        new_image = image_copy - t * grad
        new_loss = loss(new_image, data_list, coords)
        
        while new_loss > prev_loss: # Only run when new_loss > prev_loss
            new_image = image_copy - t * grad
            new_loss = loss(new_image, data_list, coords)
            t /= 2

        loss_file = open("losses_2.txt","a")
        loss_file.write("--------------------------")
        loss_file.write("t-val" + str(t) + "\n")
        loss_file.write("prev_loss:" + str(prev_loss) + "\n")
        loss_file.write("new_loss:" + str(new_loss) + "\n")
        loss_file.close()

        image_copy -= t * 2 * grad
        i += 1
        """if i == 100:
            return"""

In [ ]:
gradient_descent(emp, data_list, coords, coeffs)